In [ ]:
import glob
import os

import numpy
import pandas
import seaborn
import xarray

from src import knn

In [ ]:
seaborn.set()
figsize = (18, 5)

In [ ]:
data_store_path = 'H://Shared drives/GCM_Climate_Tool/required_files'

In [ ]:
data = pandas.read_csv('downscaled_precip.csv', parse_dates={'date': [0]})
data.set_index('date', inplace=True)

In [ ]:
future_ds_path = glob.glob(os.path.join(data_store_path, 'GCMs/Amazon__pr_day_CESM2-WACCM_ssp126_*.nc'))[0]

In [ ]:
observed_location = os.path.join(data_store_path, 'OBSERVATIONS/LLdM_AOI2/catalogo.csv')
locations = pandas.read_csv(observed_location)
lon, lat = locations[locations['CODIGO_CAT'] == 'Beni01'][['longitud', 'latitud']].values[0]

In [ ]:
gcm = xarray.open_dataset(future_ds_path)
gcm = knn.shift_longitude_from_360(gcm)
gcm = gcm.sel(lon=lon, lat=lat, method='nearest')

In [ ]:
start_date

In [ ]:
end_date

In [ ]:
start_date = data.index.min()
end_date = data.index.max()

In [ ]:
gcm = gcm.sel(time=slice(start_date, end_date))

In [ ]:
gcm_df = gcm.to_dataframe()

In [ ]:
gcm_df

In [ ]:
gcm_df['pr'] = gcm_df['pr'] * (60 * 60 * 24)

In [ ]:
results_r = pandas.read_csv(
    '../test-results/CanESM5/CanESM5_ssp126_pr_1_bootstrap.csv',
    parse_dates={'date': ['simYear', 'simMonth', 'simDay']})
results_r.set_index('date', inplace=True)

In [ ]:
d = pandas.merge(data, gcm_df, left_index=True, right_index=True, how='left')

In [ ]:
dd = pandas.merge(d, results_r['bs_value'], left_index=True, right_index=True, how='left')

In [ ]:
dd.rename({
    'historic_precip': 'python_prediction',
    'bs_value': 'R_prediction',
    'pr': 'GCM_prediction'
}, axis=1, inplace=True)

In [ ]:
pal = seaborn.color_palette("deep")
dd[['python_prediction', 'R_prediction', 'GCM_prediction']].plot(
    xlim=('2023-01-01', '2024-07-01'),
    color=[pal[0], pal[1], pal[2]],
    linewidth=1,
    figsize=figsize)

In [ ]:
# seaborn.color_palette('deep')

In [ ]:
dd['python_prediction'].hist(bins=100)

In [ ]:
dd['R_prediction'].hist(bins=100)